<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter07/Detectron2_Chapter07_PuttingAllTogether.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 07 - Putting All Together
## The dataset
Should execute the Notebook titled "Data Processing" first to process the dataset or run the following code to download the processed dataset from GitHub repository of the book.

In [ ]:
!wget -q https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/datasets/braintumors_coco.zip
!unzip -q braintumors_coco.zip

## Train models
Installation

In [ ]:
!python -m pip install \
'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-24jq1sb2
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-24jq1sb2
     |████████████████████████████████| 50 kB 2.6 MB/s 
     |████████████████████████████████| 79 kB 6.1 MB/s 
     |████████████████████████████████| 154 kB 36.0 MB/s 
     |████████████████████████████████| 1.5 MB 61.1 MB/s 
     |████████████████████████████████| 549 kB 77.4 MB/s 
     |████████████████████████████████| 117 kB 103.3 MB/s 
     |████████████████████████████████| 96 kB 5.1 MB/s 
     |████████████████████████████████| 182 kB 102.5 MB/s 
  Created wheel for detectron2: filename=detectron2-0.6-cp38-cp38-linux_x86_64.whl size=5454952 sha256=fd075c7fa4e9372a2c9adb5133adb6b79ab5c1bbab4289398432c8fe2105b607
  Stored in directory: /tmp/pip-ephem-wheel-cache-nugi1jeu/wheels/19/ac

In [ ]:
from detectron2.utils.logger import setup_logger
logger = setup_logger()

In [ ]:
from detectron2.data.datasets import register_coco_instances

In [ ]:
# Some configurations
name_ds = "braintumors_coco"
name_ds_train = name_ds + "_train"
name_ds_test = name_ds + "_test"
image_root_train = name_ds + "/train"
image_root_test = name_ds + "/test"
af = "_annotations.coco.json"
json_file_train = name_ds + "/train/" + af
json_file_test = name_ds + "/test/" + af

In [ ]:
# Register datasets
## train dataset
register_coco_instances(
    name = name_ds_train,
    metadata = {},
    json_file = json_file_train,
    image_root = image_root_train
    )
## test dataset
register_coco_instances(
    name = name_ds_test,
    metadata = {},
    json_file = json_file_test,
    image_root = image_root_test
    )


### Training configuration

In [ ]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
import pickle

In [ ]:
output_dir = "output/object_detector_chapters0607"
os.makedirs(output_dir, exist_ok=True)
output_cfg_path = os.path.join(output_dir, "cfg.pickle")
nc = 2
device = "cuda"
# Select a model
config_file_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
checkpoint_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

In [ ]:
# Create a configuration file
cfg = get_cfg()
config_file = model_zoo.get_config_file(config_file_url)
cfg.merge_from_file(config_file)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
# Download weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
# Set datasets
cfg.DATASETS.TRAIN = (name_ds_train,)
cfg.DATASETS.TEST = (name_ds_test,)
# Workers
cfg.DATALOADER.NUM_WORKERS = 2
# Solver
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.STEPS = (3000, 4000)
cfg.SOLVER.GAMMA = 0.5
cfg.SOLVER.NESTROV = False
cfg.SOLVER.MAX_ITER = 5000
# checkpoint
cfg.SOLVER.CHECKPOINT_PERIOD = 500
# anchors
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[68.33245953, 112.91302277,  89.55701886, 144.71037342,  47.77637482]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.99819939, 0.78726896, 1.23598428]]
# pixels
cfg.MODEL.PIXEL_MEAN = [20.1962, 20.1962, 20.1962]
cfg.MODEL.PIXEL_STD = [39.5985, 39.5985, 39.5985]

# Evaluation
cfg.TEST.EVAL_PERIOD = cfg.SOLVER.CHECKPOINT_PERIOD
# Classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = nc
cfg.MODEL.DEVICE = device
cfg.OUTPUT_DIR = output_dir

In [ ]:
# save configuration file for future use
with open(output_cfg_path, "wb") as f:
  pickle.dump(cfg, f, protocol = pickle.HIGHEST_PROTOCOL)

### Training

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
class BrainTumorTrainer(DefaultTrainer):
  """
  This trainer evaluate data on the `cfg.DATASETS.TEST` validation dataset every `cfg.TEST.EVAL_PERIOD` iterations.
  """
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder == None:
      output_folder = cfg.OUTPUT_DIR
    else:
      output_folder = os.path.join(cfg.OUTPUT_DIR, output_folder)
      os.makedirs(output_folder)
    # Use 
    return COCOEvaluator(dataset_name, distributed=False, output_dir=output_folder)


In [ ]:
from detectron2.engine.hooks import HookBase
import torch
import logging

class BestModelHook(HookBase):
  def __init__(self, cfg, metric="bbox/AP50", min_max="max"):
    self._period = cfg.TEST.EVAL_PERIOD
    self.metric = metric
    self.min_max = min_max
    self.best_value = float("-inf") if min_max == "max" else float("inf")
    logger = logging.getLogger("detectron2")
    logger.setLevel(logging.DEBUG)
    logger.propagate = False
    self._logger = logger
        

  def _take_latest_metrics(self):
    with torch.no_grad():
      latest_metrics = self.trainer.storage.latest()
      return latest_metrics
      
  def after_step(self):
    next_iter = self.trainer.iter + 1
    is_final = next_iter == self.trainer.max_iter
    if is_final or (self._period > 0 and next_iter % self._period == 0):
      latest_metrics = self._take_latest_metrics()
      for (key, (value, iter)) in latest_metrics.items():
        if key == self.metric:
          if (self.min_max == "min" and value < self.best_value) or (self.min_max == "max" and value > self.best_value):
            self._logger.info("Updating best model at iteration {} with {} = {}".format(iter, self.metric, value))
            self.best_value = value
            self.trainer.checkpointer.save("model_best")
            

In [ ]:
import gc
gc.collect()

16

In [ ]:
trainer = BrainTumorTrainer(cfg)
bm_hook = BestModelHook(cfg, metric="bbox/AP50", min_max="max")
trainer.register_hooks(hooks=[bm_hook])
trainer.train()

[12/31 00:24:44 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/31 00:25:24 d2.utils.events]:  eta: 1:44:02  iter: 19  total_loss: 3.571  loss_cls: 0.3393  loss_box_reg: 0.009924  loss_rpn_cls: 0.9733  loss_rpn_loc: 2.392  time: 1.2607  data_time: 0.0403  lr: 1.9981e-05  max_mem: 8302M
[12/31 00:25:48 d2.utils.events]:  eta: 1:42:52  iter: 39  total_loss: 1.659  loss_cls: 0.3964  loss_box_reg: 0.03071  loss_rpn_cls: 0.6581  loss_rpn_loc: 0.529  time: 1.2525  data_time: 0.0333  lr: 3.9961e-05  max_mem: 8302M
[12/31 00:26:15 d2.utils.events]:  eta: 1:45:00  iter: 59  total_loss: 1.429  loss_cls: 0.3705  loss_box_reg: 0.08998  loss_rpn_cls: 0.6637  loss_rpn_loc: 0.2807  time: 1.2754  data_time: 0.0356  lr: 5.9941e-05  max_mem: 8302M
[12/31 00:26:40 d2.utils.events]:  eta: 1:44:35  iter: 79  total_loss: 1.255  loss_cls: 0.2182  loss_box_reg: 0.1313  loss_rpn_cls: 0.659  loss_rpn_loc: 0.2325  time: 1.2746  data_time: 0.0341  lr: 7.9921e-05  max_mem: 8302M
[12/31 00:27:06 d2.utils.events]:  eta: 1:44:38  iter: 99  total_loss: 1.238  loss_cls: 0.2216 

## Save the best model

In [ ]:
# workaround if the next cell brings locale error
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# zip
model_best = os.path.join(cfg.OUTPUT_DIR, "model_best.pth")
!zip -r {model_best}.zip {model_best}
from google.colab import files
files.download(model_best+".zip")

  adding: output/object_detector_chapters0607/model_best.pth (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Wait for the previous download to finish and run the follwoing statement to remove the zip
!rm {model_best}.zip

## Save training information

In [ ]:
# remove the .pth file (do not run this if you would like to keep the models)
!rm {cfg.OUTPUT_DIR}/*.pth
# zip
!zip -r {cfg.OUTPUT_DIR}.zip {cfg.OUTPUT_DIR}
from google.colab import files
files.download(cfg.OUTPUT_DIR+".zip")

  adding: output/object_detector_chapters0607/ (stored 0%)
  adding: output/object_detector_chapters0607/last_checkpoint (stored 0%)
  adding: output/object_detector_chapters0607/coco_instances_results.json (deflated 71%)
  adding: output/object_detector_chapters0607/cfg.pickle (deflated 50%)
  adding: output/object_detector_chapters0607/metrics.json (deflated 78%)
  adding: output/object_detector_chapters0607/events.out.tfevents.1672446287.9bc0a3783327.1219.0 (deflated 72%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>